In [1]:
import transformers
import torch
from PIL import Image
from transformers import CLIPModel,CLIPTokenizer,CLIPProcessor
from transformers import ViTModel, ViTForImageClassification, ViTImageProcessor
import requests
import os

/home/montasir/miniconda3/envs/transformer_analysis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
torch.cuda.is_available()
device = "cuda:1" if torch.cuda.is_available() else "cpu"
device

'cuda:1'

In [51]:
checkpoint= "google/vit-base-patch16-224"
#checkpoint= "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
processor=ViTImageProcessor.from_pretrained(checkpoint)
model=ViTModel.from_pretrained(checkpoint)
model.to(device)
model = model.eval()


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
input_path = '/home/montasir/Desktop/Montasir/Summer 2024/Transformer Analysis_NIPS2023/imagenette/train/english_springer/ILSVRC2012_val_00002294.JPEG'
img_street = Image.open(input_path)

inputs_street = processor(images=img_street, return_tensors="pt")
inputs_street =inputs_street.to(device)
pixel_values_street = inputs_street['pixel_values'].to(device)
pixel_values_street


tensor([[[[-0.6549, -0.6235, -0.6000,  ..., -0.8353, -0.8510, -0.8745],
          [-0.6235, -0.5922, -0.6000,  ..., -0.8118, -0.7569, -0.7255],
          [-0.5922, -0.5922, -0.6157,  ..., -0.8667, -0.8353, -0.7961],
          ...,
          [-0.3490, -0.4824, -0.4510,  ..., -0.2863, -0.3098, -0.3333],
          [-0.4118, -0.4510, -0.3490,  ..., -0.3098, -0.3255, -0.3098],
          [-0.4588, -0.4353, -0.4118,  ..., -0.3412, -0.3176, -0.3569]],

         [[-0.7569, -0.7490, -0.7255,  ..., -0.7804, -0.8039, -0.8275],
          [-0.7490, -0.7333, -0.7098,  ..., -0.6863, -0.7020, -0.6863],
          [-0.7333, -0.7255, -0.7098,  ..., -0.8118, -0.7725, -0.7255],
          ...,
          [-0.3490, -0.4353, -0.4039,  ..., -0.2784, -0.3412, -0.3490],
          [-0.3098, -0.3725, -0.3255,  ..., -0.2941, -0.3255, -0.3020],
          [-0.3569, -0.3490, -0.3412,  ..., -0.3490, -0.3490, -0.3961]],

         [[-0.7490, -0.7490, -0.7490,  ..., -0.6392, -0.7020, -0.7647],
          [-0.7569, -0.7490, -

In [39]:
with torch.no_grad():
    output =model(pixel_values = pixel_values_street,output_hidden_states=True)
output

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.2847,  0.9427,  1.5724,  ..., -0.8000, -0.3915,  0.2022],
         [-0.5942,  0.9252,  1.5284,  ...,  0.0354,  0.3504, -0.4656],
         [-1.0470,  1.1819,  2.0243,  ...,  0.0098,  0.5940,  0.0081],
         ...,
         [-0.7913,  1.5144,  0.7939,  ..., -0.2541,  0.5426,  0.3481],
         [-0.8147,  1.0578,  0.1884,  ..., -0.5435,  0.3623,  0.1273],
         [-1.1294,  1.1140,  0.3807,  ..., -0.3310,  0.8359, -0.1823]]],
       device='cuda:1'), pooler_output=tensor([[-4.5920e-01, -3.9348e-01, -6.0311e-01,  1.4339e-01,  8.1634e-02,
          5.1593e-01,  1.1250e-01,  3.8625e-01,  2.3020e-01,  6.9805e-02,
         -7.4941e-02, -5.6041e-01,  2.5387e-01, -5.3431e-01, -7.4834e-01,
          4.9151e-01,  1.0472e-01,  6.3602e-01,  5.8541e-02, -8.6976e-02,
         -6.2308e-01,  4.3769e-01,  8.1900e-02, -4.3965e-01, -6.6768e-01,
         -2.4553e-01,  1.9601e-01, -2.1312e-01, -1.7240e-01,  8.1679e-01,
         -7.7655e-02, -8.0176e

In [36]:
output['hidden_states'][11]

tensor([[[ 3.0391,  0.5210,  2.0633,  ..., -5.1688, -4.2767,  1.7423],
         [-2.9999,  1.5158,  6.0074,  ..., -2.0337,  5.2446, -6.9205],
         [-5.1570,  1.9435,  9.8021,  ..., -0.6604,  4.2677, -3.4356],
         ...,
         [-3.3529,  2.6084, -1.9761,  ..., -2.5863,  4.5115,  1.0607],
         [-6.1727,  2.7064, -3.0866,  ..., -1.3029,  1.8416, -1.8960],
         [-7.9776,  2.7544, -2.7417,  ..., -2.5574,  7.9432, -3.8450]]],
       device='cuda:1')

In [60]:
####PGD with counter

import torch
import torch.nn.functional as F

def optimize_embeddings_pgd(model, cur_input, target_emb, learning_rate, epsilon, l2_dist_threshold, cosine_sim_threshold):
    """
    Adjusts initial input to match target embedding using Projected Gradient Descent.
    Args:
        model (torch.nn.Module): Model for generating embeddings.
        cur_input (torch.Tensor): Input tensor to be optimized.
        target_emb (torch.Tensor): Target embedding to match.
        learning_rate (float): Learning rate for gradient descent.
        epsilon (float): Epsilon value for controlling perturbations in PGD.
        l2_dist_threshold (float): Threshold for squared L2 distance.
        cosine_sim_threshold (float): Threshold for cosine similarity.
    Returns:
        torch.Tensor: Optimized input tensor.
        list: L1 distances over iterations.
        list: Cosine similarities over iterations.
        list: Losses over iterations.
    """
    org_input = cur_input.clone()
    

    squared_l2_distance = float('inf')
    cosine_sim_arr = []
    loss_arr = []
    l1_dist_arr = []
    exit_counter = 0
    squared_l2_distance_arr=[]

    iteration_count = 0
    cosine_sim = 0
    squared_l2_distance =float('inf')
    
    while squared_l2_distance >= l2_dist_threshold or cosine_sim <= cosine_sim_threshold:
        cur_input = cur_input.clone().detach().requires_grad_(True)
        
        cur_emb= model.forward(cur_input).last_hidden_state
        #embedding of CLS token
        cur_emb =cur_emb[0,0]

        loss = F.mse_loss(target_emb, cur_emb)
        loss_arr.append(loss.item())

        cur_input.grad = None
        loss.backward(retain_graph=True)
        grad = cur_input.grad

        updated_input = cur_input - learning_rate * grad

    
        projected_input = cur_input + torch.clamp(updated_input - cur_input, -epsilon, epsilon)
       
        with torch.no_grad():
            updated_emb= model.forward(projected_input).last_hidden_state
        updated_emb = updated_emb[0,0]

        squared_l2_distance = torch.sum((target_emb - updated_emb)**2).item()

        updated_l1_dist = torch.sum(torch.abs(projected_input.detach() - org_input)).item()
        cosine_sim = F.cosine_similarity(target_emb.unsqueeze(0), updated_emb.unsqueeze(0))

        l1_dist_arr.append(updated_l1_dist)
        cosine_sim_arr.append(cosine_sim.detach().cpu().item())
        squared_l2_distance_arr.append(squared_l2_distance)
        
        cur_input = projected_input
        iteration_count += 1
    

    return cur_input.detach(), l1_dist_arr, cosine_sim_arr, loss_arr,squared_l2_distance_arr,iteration_count,cosine_sim,squared_l2_distance,




In [61]:
def inverse_normalize():
    """
    Returns an inverse normalization transformation.
    Returns:
        transforms.Normalize: Transformation to apply inverse normalization.
    """
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    return transforms.Normalize(mean=[-m/s for m, s in zip(mean, std)], std=[1/s for s in std])


In [62]:
from PIL import Image
from torchvision import transforms

image_path = '/home/montasir/Desktop/Montasir/Summer 2024/Transformer Analysis_NIPS2023/imagenette/train/parachute/n03888257_1137.JPEG'
image = Image.open(image_path)

input_image = processor(image,return_tensors ="pt")['pixel_values'].to(device) 

target_img_path = '/home/montasir/Desktop/Montasir/Summer 2024/Transformer Analysis_NIPS2023/imagenette/train/tench/ILSVRC2012_val_00009379.JPEG'
image2 = Image.open(target_img_path)

target_image = processor(image2,return_tensors ="pt")['pixel_values'].to(device) 

with torch.no_grad():
    target_embedding= model(pixel_values =target_image).last_hidden_state
target_embedding = target_embedding[0,0]


In [ ]:
from torchvision.utils import save_image

save_image(input_image, './input_image.png')

#optimized_image,_,_,_, = optimize_embeddings(model, input_image, target_embedding, learning_rate=0.09, l2_dist_threshold=80, cosine_sim_threshold=.85)
optimized_image,_,_,_,_,_,_,_ = optimize_embeddings_pgd(model, input_image, target_embedding, learning_rate=0.9,epsilon= 0.02,l2_dist_threshold=16, cosine_sim_threshold=.95)
optimized_image_inv = inverse_normalize()(optimized_image)
save_image(optimized_image_inv, './optimized_image.png')


